# Set-RscMssqlInstance
This will allow you to assign an SLA to a MSSQL Instance.

# Workflow
To set information about the SQL Server Database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the host id that the SQL Server is installed on. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Set-RscMssqlInstance** to make a change to the configuration of Rubrik for a specific MSSQL Instance. 
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [ ]:
Set-Location '/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC

## Get the Cluster ID

In [ ]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host Information

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [ ]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [ ]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance
$RscMssqlDatabase

## Change the configuration of Rubrik for the database
### Set DO NOT PROTECT on a database
This is used to tell Rubrik to not backup a database

In [ ]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -DoNotProtect -ExistingSnapshotRetention RETAIN_SNAPSHOTS

### Set "Clear Existing assignment"
This is used to tell Rubrik clear the direct assignment and revert back to the SLA assignment from a higher level. Effectively reverses the DO NOT PROTECT setting. 

In [ ]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -ClearExistingProtection

### Set an SLA to the database
This should only be used when the database needs a specific SLA that differs from the Host/Instance assigned SLA

In [ ]:
$RscSlaDomain = Get-RscSlaDomain -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain

As part of the assigning of the SLA, there are various options to set in addition to the SLA Domain.
#### Configuring Log Backups
Enabling log backups ensures that Point In Time Recovery is available for a database. 

In [ ]:
$RscSlaDomain = Get-RscSlaDomain -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups -UseSLALogConfig

In [ ]:
$RscSlaDomain = Get-RscSlaDomain -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups -logBackupFrequencyInSeconds 3600 -logRetentionHours 168

#### Configure Host Log Retention

In [ ]:
$RscSlaDomain = Get-RscSlaDomain -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups -EnableHostLogRetention -HostLogRetentionInSeconds 3600

#### Disable Host Log Retention

In [ ]:
$RscSlaDomain = Get-RscSlaDomain -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups

#### Set Pre-Script

In [ ]:
$PreBackupScriptPath = "c:\rubrik\scripts\dosomething.bat"
$PreBackupScriptTimeoutMs = 5000
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -PreBackupScriptErrorAction "SCRIPT_ERROR_ACTION_ABORT" `
    -PreBackupScriptPath $PreBackupScriptPath `
    -PreBackupScriptTimeoutMs $PreBackupScriptTimeoutMs

#### Set Post-Script

In [ ]:
$PostBackupScriptPath = "c:\rubrik\scripts\dosomething.bat"
$PostBackupScriptTimeoutMs = 5000
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -PostBackupScriptErrorAction "SCRIPT_ERROR_ACTION_CONTINUE" `
    -PostBackupScriptPath $PostBackupScriptPath `
    -PostBackupScriptTimeoutMs $PostBackupScriptTimeoutMs

#### Remove Pre Scipt

In [ ]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -RemovePreBackupScript

#### Remove Post Scipt

In [ ]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -RemovePostBackupScript

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc